# Scraping data from Reddits Wallstreetbets subreddit

In [144]:
from datetime import datetime
import pandas as pd 
from pandas import DataFrame 
import praw
from praw.models import MoreComments 
import re
import requests     
from textblob import TextBlob 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk 
import warnings 
warnings.filterwarnings("ignore")

In [145]:
# common words that are also stock tickers or definetly uneccesary. Excluding them from the analysis since the context is hard to fig
exclude = ['VERY', 'A', 'B', 'GO', 'ARE', 'ON', 'FOR', 'THE', 'TO', ' ', 'SO', 'IT','AT', 'BE', 'OR', 'SO', 'ALL', 'HAS', 'BY', 'CAN', 'AN', 'OUT', 'NOW']
now = datetime.now() # set time to now

In [146]:
# setting up redit client
reddit = praw.Reddit(
  client_id = "dlY27DaxJQaL5Q",
  client_secret = "z-MStmsM-inT4-XJmeGtovN05XCEgw",
  user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
)

In [147]:
# This function is used to create a frequency table of the scraped comments
def frequency_table_comments(clean):
    for (index, row) in clean.iterrows(): # iterate over dataframe
  # titles 
        title = row['comments'].upper() # get title in lowercase
        title = regex.sub('', title)  # clean with reges
        title_words = title.split(' ') # split titles at whitespace
    for words in title_words:
        if x in exclude: # common words that are also stock tickers or definetly uneccesary. Excluding them from the analysis since the context is hard to figure out
            pass
        elif x in word_dict:
            word_dict[x] += 1
        else:
            word_dict[x] = 1
    return pd.DataFrame.from_dict(list(word_dict.items())).rename(columns = {0:"Term", 1:"Frequency"})

In [148]:
# defining funtion to append comments to a list and creating a dataframe from them
def appending_to_list(submission):
    comments= []
    for top_level_comment in submission.comments[:-1]: #leaving out the last comment, since it creates an error
        comments.append(top_level_comment.body) # append comment to list
# return dataframe of the list
    return DataFrame(comments,columns=['comments'])

# Step 1: Scrape posts into dataframe. 
Reddit divides posts within a subreddit (think subforum) into multiple categories, such as new, hot and top posts.
The cell below scrapes these exact categories in the 'wallstreetbets' subreddit

In [149]:
# getting posts into a dataframe
df = [] # define empty list that will hold dictionarys
#scraper =     
for post in reddit.subreddit('wallstreetbets').hot(limit=500): # call wallstreetbets subreddit "hot" section and get first 1000 posts
    content = {  # create dictionary for results
    "title" : post.title, # store title
    "text" : post.selftext # store text of the post
    }
    df.append(content) # append dataframe
for post in reddit.subreddit('wallstreetbets').new(limit=2000): # call wallstreetbets subreddit "new" section and get first 1000 posts
    content = {  # create dictionary for results
    "title" : post.title, # store title
    "text" : post.selftext # store text of the post
    }
    df.append(content) # append dataframe
for post in reddit.subreddit('wallstreetbets').top(limit=500): # call wallstreetbets subreddit "top" section and get first 1000 posts
    content = {  # create dictionary for results
    "title" : post.title, # store title
    "text" : post.selftext # store text of the post
    }
    df.append(content) # append dataframe
    df_posts = pd.DataFrame(df) # convert to pandas dataframe

# Step 2: Create frequency table of unique words and the number of their mentions 

In [150]:
regex = re.compile('[^a-zA-Z ]') # remove everything thats not a letter or space aka numbers and emojis
word_dict = {} # create dictionary
for (index, row) in df_posts.iterrows(): # iterate over dataframe
  # titles 
    title = row['title'].upper() # get title in lowercase
    title = regex.sub('', title)  # clean with reges
    title_words = title.split(' ') # split titles at whitespace
  # content
    content = row['text'].upper() # get text from post in lowercase
    content = regex.sub('', content) # clean with regex
    content_words = content.split(' ') # split titles at whitespace
  # combine titles and comments
    words = title_words + content_words
    for x in words:
        if x in exclude:
            pass
        elif x in word_dict:
            word_dict[x] += 1
        else:
            word_dict[x] = 1
posts_freq= pd.DataFrame.from_dict(list(word_dict.items())).rename(columns = {0:"Term", 1:"Frequency"})

,Term,Frequency
0,WHAT,560
1,YOUR,486
2,MOVES,62
3,TOMORROW,72
4,MAY,365
...,...,...
14465,LEGION,1
14466,ORGANISATION,1
14467,STANDALONES,1
14468,ITGODSPEED,1


## We now have a frequency table of the most often used words in the top 500 hot posts
### Next up is scraping the top level comments from the daily discussion thread


In [151]:
# get comments of daily discussion thread
url = ("https://www.reddit.com/r/wallstreetbets/comments/n9th6n/daily_discussion_thread_for_"+
str(now.strftime("%B")).lower() + "_"+  # constructing the link with daytime function
str(now.strftime("%d")).lower() +"_" +  # since this post is created new on a daily basis
str(now.strftime("%Y")).lower() + "/")  # Month, day, year

submission = reddit.submission(url=url)

In [152]:
# append these comments to a list
daily_comments= [] # create empty list
for top_level_comment in submission.comments[:-1]: #leaving out the last comment, since it creates an error
        daily_comments.append(top_level_comment.body) # append comment to list
len(daily_comments)

# create dataframe of the comments
df_comments = DataFrame(daily_comments,columns=['comments'])

In [153]:
df_comments = appending_to_list(submission)

In [154]:
# calling cleaning/splitting function to create frequency table
comments_freq = frequency_table_comments(df_comments)

## Scraping the "what are your moves" thread

In [155]:
# get comments of daily discussion thread
url = ("https://www.reddit.com/r/wallstreetbets/comments/n9eiyu/what_are_your_moves_tomorrow_"+
str(now.strftime("%B")).lower() + "_"+  # constructing the link with daytime function
str(now.strftime("%d")).lower() +"_" +  # since this post is created new on a daily basis
str(now.strftime("%Y")).lower() + "/")  # Month, day, year
submission = reddit.submission(url=url)

In [156]:
df_comments_move = appending_to_list(submission)

In [157]:
# calling cleaning/splitting function from above
moves_freq = frequency_table_comments(df_comments_move)

In [158]:
# merging all frequency tables on the term, performing an inner join 
merged_table = pd.merge(pd.merge(posts_freq,comments_freq,on='Term', how='left'),moves_freq,on='Term', how = 'left')
# replace NA#s with zeros
merged_table = merged_table.fillna(0)

In [159]:
# calculating overall frequency in a new column
merged_table['frequency']=merged_table['Frequency_x'] + merged_table['Frequency_y'] + merged_table['Frequency']
# drop unnecessary frequency columns, only keeping the main one
merged_table = merged_table.drop(['Frequency_x', 'Frequency_y', 'Frequency'], axis = 1) 

In [160]:
# importing a tickerlist from the nasdaq
ticker_df = pd.read_csv('tickers.csv').rename(columns = {"Symbol":"Term", "Name":"Company_Name"})

In [161]:
# merging tickerlist with mentions on reddit, inner join, so we will loose all words that are not tickers and all tickers that arent mentioned
stonks_df = pd.merge(merged_table, ticker_df, on="Term")

In [162]:
# exporting data to csv
stonks_df.to_csv('frequency_of_tickers_' + str(now.strftime("%d"))+'_'+str(now.strftime("%m"))+ '_'+ str(now.strftime("%H"))+'.csv', index=False)

***

### Adding sentiment to full comments

In [163]:
# get list of tickers that are in the above dataframe
tickers = ticker_df['Term'].tolist()

['A',
 'AA',
 'AAC',
 'AACG',
 'AACQ',
 'AACQU',
 'AACQW',
 'AAIC',
 'AAIC^B',
 'AAIC^C',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABGI',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABR',
 'ABR^A',
 'ABR^B',
 'ABR^C',
 'ABST',
 'ABT',
 'ABTX',
 'ABUS',
 'AC',
 'ACA',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACB',
 'ACBAU',
 'ACBI',
 'ACC',
 'ACCD',
 'ACCO',
 'ACEL',
 'ACER',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACI',
 'ACIC',
 'ACII',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACND',
 'ACOR',
 'ACP',
 'ACQRU',
 'ACR',
 'ACR^C',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACST',
 'ACTC',
 'ACTCU',
 'ACTCW',
 'ACTDU',
 'ACTG',
 'ACU',
 'ACV',
 'ACVA',
 'ACY',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADER',
 'ADERU',
 'ADERW'

In [164]:
# using Vader to analyse social media sentiment better than with nltk
sia = SentimentIntensityAnalyzer()

In [165]:
def sentiment(text):
    try:
        scores = sia.polarity_scores(text)
        return  scores['compound']
    except: return none

In [166]:
# add polarity column to posts
df_posts['polarity'] = df_posts['text'].apply(sentiment)
# drop title column
df_posts = df_posts.drop(['title'], axis=1)
# add polarity column 
df_comments['polarity'] = df_comments['comments'].apply(sentiment)
df_comments_move['polarity'] = df_comments_move['comments'].apply(sentiment)

In [167]:
# creating a list of the three dataframes to make iterating for sentiment extraction easier
list_df = [df_posts, df_comments, df_comments]

In [168]:
# extracting sentiment and associated ticker out of the dataframe
sentiment_to_df= [] # list for sentiment score
tickers_to_df = [] # list for tickers

for dataframe in list_df: 
    for index, row in dataframe.iterrows(): # for every row in dataframe
        title_words = row[0].split(' ') # split comment at whitespace and put into list
        for word in title_words:
            if word in tickers: 
                tickers_to_df.append(word) # append the word to the word list
                sentiment_to_df.append(row[1]) # append the sentiment to the sentiment list

In [176]:
# create sentiment dataframe 
sentiment_df1 = pd.DataFrame(list(zip(tickers_to_df, sentiment_to_df)),
              columns =['Term', 'Polarity'])

In [170]:
# group table by ticker, get mean of sentiment, create new dataframe with new index
sentiment_table=sentiment_df1.groupby(['Term']).mean()
sentiment_table['Term'] = sentiment_table.index
sentiment_table=sentiment_table.reset_index(drop=True)

In [171]:
# export to csv
sentiment_table.to_csv('sentiment_of_tickers_' + str(now.strftime("%d"))+'_'+str(now.strftime("%m"))+ '_'+ str(now.strftime("%H"))+'.csv', index=False)